# Análise de aluguéis - São Paulo - Data Prep + Regressão linear

Este notebook tem como objetivo realizar uma regressão para prever o preço de imóveis da cidade de SP. Estou iniciando na área de modelagem de dados e esta analise é baseada no que conheço e no que aprendi com outros notebooks deste site.
Vou melhorar com o tempo este notebook. 
Aceito sujestões de melhoria no código tbm ;)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm

In [ ]:
df = pd.read_csv("../input/brasilian-houses-to-rent/houses_to_rent_v2.csv")
df.head()
df = df.loc[df['city'] == 'São Paulo']
df.head()

Analise da base e data prep com as colunas que irei trabalhar.

In [ ]:
df.isnull().sum()

Após verificar que não há registros nulos na base, vou eliminar as colunas que não serão utilizadas e renomear as demais.

In [ ]:
df = df.drop(['city'],axis=1)
df = df.drop(['hoa (R$)'],axis=1)
df = df.drop(['total (R$)'],axis=1)
df = df.drop(['property tax (R$)'],axis=1)
df = df.drop(['fire insurance (R$)'],axis=1)
df = df.rename({'rent amount (R$)': 'rent_amount',}, axis=1) 
df = df.rename({'parking spaces': 'parking_spaces',}, axis=1) 
df.head()

Transformando os dados categóricos em numéricos.

In [ ]:
def categ_code(df, coluna):
  df[coluna] = pd.Categorical(df[coluna])
  return df[coluna].cat.codes
df['animal'] = categ_code(df, 'animal')
df['furniture'] = categ_code(df, 'furniture')
df = df.replace({'floor':{'-': 0 }})
df['floor'] = pd.to_numeric(df['floor'])
df.dtypes

Análise de correlação da base atual. É possivel observar que não há grandes relações entre a variável objetivo *rent_amount* e as demais.

In [ ]:
correlacao = df.corr()
plt.figure(figsize=(8, 7))
sns.heatmap(correlacao)

In [ ]:
g = sns.scatterplot(x="rent_amount", y="area",data=df)

Convertemos a variavel objetivo e a area em log na base 10.
Acredito que há grande relação entre essas variaveis porem a variação de valores pode esta interferindo.

In [ ]:
df['rent_amount_log'] =  np.log1p(df['rent_amount'])
df['area_log'] = np.log1p(df['area'])
g = sns.scatterplot(x="rent_amount_log", y="area_log",data=df)

Nova correlação após a normalização dos dados.

In [ ]:
correlacao = df.corr()
plt.figure(figsize=(8, 7))
sns.heatmap(correlacao)

Com a melhora na correlação vamos aplicar a regressão.

In [ ]:
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

X = df.drop(['rent_amount','floor','area'],axis=1).values
y = df['rent_amount'].values
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size= 0.2)

Análise dos resultados:

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)
print("Score treino: " + str(model.score(X_train,y_train)))

In [ ]:
model = LinearRegression()
model.fit(X_test, y_test)
model.coef_
print("Score teste: " + str(model.score(X_test,y_test)))

Com base nos resultados podemos dizer que o modelo esta acertando tanto na base teste quanto na treino e poderia ser aplicado em uma base real.
Aplicando os valores na base:

In [ ]:
df['predict'] = model.predict(X)
df.head()

# Conclusão da análise

Podemos observar que o modelo possui uma boa assertividade e com valores aproximados entre treino e teste, porém está aplicando valores negativos na base. Vou analisar outros tipos de modelo para melhorar a análise.